# Problem A: Implementacja algorytmu selekcji turniejowej

## Prompt i modyfikowany kod

In [ ]:
class PopulationPerm:
    def __init__(self, population_size, vrproblem, mutation_chance=0.05, crossover_rate=0.6):
        self.pop = []
        self.eval = np.array([])
        self.best_route = np.array([])
        self.shortest_dist = float('inf')
        self.pop_size = population_size
        self.vrproblem = vrproblem
        self.mutation_chance = mutation_chance
        self.crossover_rate = crossover_rate

    def gen_population(self):
        for _ in range(self.pop_size):
            route = np.random.permutation(self.vrproblem.get_sol_len())
            self.pop.append(route.astype(int))
        self.best_route = np.array([i for i in range(0, self.vrproblem.get_depot_num())])

    def eval_population(self):
        self.eval = np.array([self.vrproblem.calc_cost(route) for route in self.pop])

    def select_roulette(self):
        absolute_eval = np.abs(self.eval - self.eval.max() - 0.1)
        eval_div = absolute_eval / absolute_eval.sum()
        eval_div = eval_div.cumsum()
        new_pop = []
        for _ in range(self.pop_size):
            rand_val = np.random.random()
            for j in range(self.pop_size):
                if eval_div[j] >= rand_val:
                    new_pop.append(self.pop[j])
                    break
        self.pop = new_pop

    def select_tournament(self, tournament_size):
        selected_population = []
        pop_size = self.pop_size
        for _ in range(pop_size):
            tournament_indices = random.sample(range(pop_size), tournament_size)
            tournament_fitness_values = [self.eval[i] for i in tournament_indices]
            selected_index = tournament_indices[np.argmax(tournament_fitness_values)]
            selected_population.append(self.pop[selected_index])
        self.pop = selected_population

    def mutate_one(self, entity):
        entity_length = len(entity)
        index_a = np.random.randint(0, entity_length)
        index_b = np.random.randint(0, entity_length)
        entity[index_a], entity[index_b] = entity[index_b], entity[index_a]
        return entity

    def mutate_inverse(self, entity):
        entity_length = len(entity)
        start = np.random.randint(0, entity_length)
        end = np.random.randint(0, entity_length)
        if start > end:
            start, end = end, start
        while start < end:
            entity[start], entity[end] = entity[end], entity[start]
            start += 1
            end -= 1
        return entity

    def crossover_ox(self,parent1, parent2):
        length = len(parent1)
        start = np.random.randint(0, length)
        end = np.random.randint(start, length)
        child = [None] * length
        child[start:end+1] = parent1[start:end+1]
        idx_child = (end + 1) % length
        idx_parent2 = (end + 1) % length
        while None in child:
            if child[idx_child] is None:
                if parent2[idx_parent2] not in child:
                    child[idx_child] = parent2[idx_parent2]
                    idx_child = (idx_child + 1) % length
            idx_parent2 = (idx_parent2 + 1) % length
        return np.array(child)

    def crossover_population(self):
        num_parents = int(self.crossover_rate * self.pop_size)
        parents_indices = np.random.choice(self.pop_size, size=num_parents, replace=False)
        for i in range(0, num_parents, 2):
            parent1 = self.pop[parents_indices[i]]
            parent2 = self.pop[parents_indices[i + 1]]
            child1 = self.crossover_ox(parent1, parent2)
            child2 = self.crossover_ox(parent2, parent1)
            self.pop[parents_indices[i]] = child1
            self.pop[parents_indices[i + 1]] = child2

    def mutate_population(self):
        for i in range(self.pop_size):
            rand_val = np.random.random()
            if rand_val < self.mutation_chance:
                self.pop[i] = self.mutate_one(self.pop[i])
                self.pop[i] = self.mutate_inverse(self.pop[i])

    def get_best(self):
        for it, dist in enumerate(self.eval):
            if dist < self.shortest_dist:
                self.shortest_dist = dist
                self.best_route = self.pop[it]
        return self.shortest_dist

    def print_best(self):
        self.get_best()
        print(self.best_route)
        print(self.shortest_dist)
        self.vrproblem.print_routes(self.best_route)
        self.vrproblem.plot(self.best_route)

Modify the select_roulette function to implement a tournament selection process. The new function should create a new population that replaces the old one, maintaining the same size. This function should be a part of the PopulationPerm class I have provided.

## Odpowiedzi

### Llama3

In [ ]:
def select_tournament(self):
    new_pop = []
    for _ in range(self.pop_size):
        # Select 3 parents for each child
        parents = np.random.choice(self.pop, size=3, replace=False)
        parent1 = parents[0]
        parent2 = parents[1]
        parent3 = parents[2]

        # Calculate the fitness of each parent
        fit1 = self.vrproblem.calc_cost(parent1)
        fit2 = self.vrproblem.calc_cost(parent2)
        fit3 = self.vrproblem.calc_cost(parent3)

        # Select the fittest parent as the child's parent
        if fit1 <= fit2 and fit1 <= fit3:
            child = parent1
        elif fit2 <= fit1 and fit2 <= fit3:
            child = parent2
        else:
            child = parent3

        new_pop.append(child)

    self.pop = new_pop

In [ ]:
def select_tournament(self):
    population_size = len(self.pop)
    tournament_size = int(population_size * 0.1)
    
    selected_routes = []
    for _ in range(int(population_size / 2)):
        tournament_routes = np.random.choice(population_size, size=tournament_size, replace=False)
        best_route = min(tournament_routes, key=lambda route: self.eval[route])
        selected_routes.append(best_route)

    new_pop = []
    for i in range(population_size):
        if i % 2 == 0:
            new_pop.append(selected_routes[i // 2])
        else:
            new_pop.append(self.pop[i])

    self.pop = new_pop

In [ ]:
def select_tournament(self):
    population_size = len(self.pop)
    tournament_size = int(population_size * 0.1)

    selected_routes = []
    for _ in range(int(population_size / 2)):
        tournament_routes = np.random.choice(population_size, size=tournament_size, replace=False)
        best_individual = min(tournament_routes, key=lambda route: self.eval[route])
        selected_routes.append(self.pop[best_individual])

    new_pop = []
    for i in range(population_size):
        if i % 2 == 0:
            new_pop.append(selected_routes[i // 2])
        else:
            new_pop.append(self.pop[i])

    self.pop = new_pop

In [ ]:
def selection_tournament(self):
    tour_size = int(len(self.pop) * 0.1)
    new_pop = []
    
    for _ in range(int(len(self.pop) / 2)):
        parent1 = np.random.choice(self.pop, size=1)[0]
        parent2 = np.random.choice([i for i in self.pop if i != parent1], size=1)[0]
        
        if self.vrproblem.calc_cost(parent1) < self.vrproblem.calc_cost(parent2):
            new_pop.append(parent1)
        else:
            new_pop.append(parent2)
    
    return new_pop

In [ ]:
def select_tournament(self):
    tournament_size = int(0.1 * self.pop_size)
    new_pop = []
    
    for _ in range(self.pop_size // 2):
        parents = np.random.choice(self.pop_size, size=tournament_size, replace=False)
        winner_route = None
        winner_cost = float('inf')
        
        for parent in parents:
            cost = self.vrproblem.calc_cost(self.pop[parent])
            if cost < winner_cost:
                winner_cost = cost
                winner_route = self.pop[parent]
        
        new_pop.append(winner_route)
    
    new_pop += self.pop[self.pop_size // 2:]
    self.pop = new_pop

### CodeLlama

In [ ]:
def select_tournament(population, tournament_size):
    # Select the fittest individuals from the population based on the tournament selection method
    selected_individuals = []
    for i in range(len(population)):
        # Select a random subset of individuals from the population
        random_indices = np.random.choice(len(population), size=tournament_size, replace=False)
        # Evaluate the fitness of each individual in the selected subset
        fitnesses = [evaluate_fitness(population[i]) for i in random_indices]
        # Select the individual with the highest fitness score
        best_index = np.argmax(fitnesses)
        selected_individuals.append(random_indices[best_index])
    return selected_individuals

In [ ]:
def tournament_selection(self):
    tournament_rate = 0.01
    num_participants = int(tournament_rate * self.pop_size)
    participants_indices = np.random.choice(self.pop_size, size=num_participants, replace=False)
    best_index = np.argmin(self.eval[participants_indices])
    return participants_indices[best_index]

In [ ]:
def tournament_selection():
    pass
    # TODO: implement tournament selection
    # return the best candidate from the tournament

### Dolphin-Mixtral

In [ ]:
def select_tournament(self, tournament_size=5):
    # Create new population through tournament selection
    absolute_eval = np.abs(self.eval - self.eval.max() - 0.1)
    num_parents = int(len(self.pop) * self.crossover_rate)
    parents_indices = np.random.choice(range(len(self.pop)), size=num_parents, replace=False)
    new_pop = []
    for _ in range(len(self.pop)):
        # Tournament selection
        tournament = absolute_eval[np.random.choice(range(len(absolute_eval)), tournament_size, replace=False)]
        winner_index = np.argmin(tournament)
        new_pop.append(self.pop[winner_index])
    self.pop = new_pop

    # Replace old population with the new one
    for i in range(len(new_pop)):
        self.pop[i] = new_pop[i]
    self.eval = np.array([self.vrproblem.calc_cost(route) for route in self.pop])

In [ ]:
def select(self):
    tournament_size = len(self.pop) // 2
    winners = []
    for _ in range(tournament_size):
        i1, i2 = np.random.randint(0, len(self.pop)), np.random.randint(0, len(self.pop))
        if self.pop[i1] < self.pop[i2]:
            winners.append(self.pop[i1])
        else:
            winners.append(self.pop[i2])
    return np.array(winners)

def tournament_selection(self):
    tournament = self.select()
    population = self.pop
    new_population = []
    for _ in range(len(population)):
        new_population.append(tournament.select())
    return np.array(new_population)

In [ ]:
def select_tournament(self):
        absolute_eval = np.abs(self.eval - self.eval.max() - 0.1)
        eval_div = absolute_eval / absolute_eval.sum()
        eval_div = eval_div.cumsum()
        new_pop = []
        for _ in range(self.pop_size):
            tournament_winners = np.random.choice(range(self.pop_size), size=10, replace=False)
            best_fitness = self.eval[np.argmax([self.eval[i] for i in tournament_winners])]
            best_route = self.pop[np.argmax([self.eval[i] for i in tournament_winners])]
            if np.random.random() < (best_fitness / np.mean(absolute_eval)):
                new_pop.append(best_route)
        self.pop = new_pop

In [ ]:
def select_tournament(self):
        tournament_size = int(np.ceil(2 * np.log10(self.pop_size)))
        new_pop = []
        for _ in range(self.pop_size):
            subset = self._sample_subset(tournament_size)
            best_idx = subset[np.argmin([self.eval[i] for i in subset])]
            new_pop.append(self.pop[best_idx])
        self.pop = new_pop

def _sample_subset(self, size):
        return np.random.choice(range(self.pop_size), size, replace=False)

### ChatGPT

In [ ]:
def select_tournament(self):
    new_pop = []
    for _ in range(self.pop_size):
        tournament = np.random.choice(self.pop_size, self.tournament_size, replace=False)
        best_idx = min(tournament, key=lambda idx: self.eval[idx])
        new_pop.append(self.pop[best_idx])
    self.pop = new_pop

In [ ]:
def select_tournament(self, tournament_size):
    new_pop = []
    for _ in range(self.pop_size):
        tournament_contestants = np.random.choice(self.pop, size=tournament_size, replace=False)
        tournament_evals = [self.vrproblem.calc_cost(route) for route in tournament_contestants]
        winner_index = np.argmin(tournament_evals)
        new_pop.append(tournament_contestants[winner_index])
    self.pop = new_pop

In [ ]:
def select_tournament(self, tournament_size):
    new_pop = []
    for _ in range(self.pop_size):
        tournament_contestants_indices = np.random.choice(len(self.pop), size=tournament_size, replace=False)
        tournament_contestants = [self.pop[i] for i in tournament_contestants_indices]
        tournament_evals = [self.vrproblem.calc_cost(route) for route in tournament_contestants]
        winner_index = np.argmin(tournament_evals)
        new_pop.append(tournament_contestants[winner_index])
    self.pop = new_pop

In [ ]:
def select_tournament(self, tournament_size):
    new_pop = []
    for _ in range(self.pop_size):
        # Randomly select individuals for the tournament
        tournament_indices = np.random.choice(self.pop_size, size=tournament_size, replace=False)
        tournament_evals = self.eval[tournament_indices]
        # Find the index of the best individual in the tournament
        best_index = tournament_indices[np.argmin(tournament_evals)]
        # Add the best individual to the new population
        new_pop.append(self.pop[best_index])
    self.pop = new_pop

### Gemini

In [ ]:
def select_tournament(self, tournament_size=2):
    """
    Performs tournament selection to select parents for the next generation.

    Args:
        tournament_size (int, optional): Number of individuals competing in each tournament. Defaults to 2.

    Returns:
        list: A list of indices of the selected parents.
    """
    new_pop = []
    for _ in range(self.pop_size):
        # Select individuals for the tournament
        candidates = np.random.choice(self.pop_size, size=tournament_size, replace=False)
        # Find the winner of the tournament (lowest fitness)
        winner_idx = np.argmin(self.eval[candidates])
        new_pop.append(self.pop[candidates[winner_idx]])
    self.pop = new_pop

In [ ]:
def select_tournament(self, tournament_size=2):
    """
    Performs tournament selection to select parents for the next generation.

    Args:
        tournament_size (int, optional): Number of individuals competing in each tournament. Defaults to 2.

    Returns:
        list: A list of indices of the selected parents.
    """
    new_pop = []
    for _ in range(self.pop_size):
        # Select individuals for the tournament
        candidates = np.random.choice(self.pop_size, size=tournament_size, replace=False)
        # Find the winner of the tournament (lowest fitness)
        winner_idx = np.argmin(self.eval[candidates])
        new_pop.append(self.pop[candidates[winner_idx]])
    return new_pop

In [ ]:
def select_tournament(self, tournament_size=2):
    new_pop = []
    for _ in range(self.pop_size):
        # Select random individuals for tournament
        candidates = np.random.choice(self.pop_size, size=tournament_size, replace=False)
        # Find the individual with the best fitness in the tournament
        best_index = np.argmin(self.eval[candidates])
        new_pop.append(self.pop[candidates[best_index]])
    self.pop = new_pop

# Problem B - Optymalizacja operatora krzyżowania

## Prompt i modyfikowany kod

In [ ]:
class PopulationPerm:
    def __init__(self, population_size, vrproblem, mutation_chance=0.05, crossover_rate=0.6):
        self.pop = []
        self.eval = np.array([])
        self.best_route = np.array([])
        self.shortest_dist = float('inf')
        self.pop_size = population_size
        self.vrproblem = vrproblem
        self.mutation_chance = mutation_chance
        self.crossover_rate = crossover_rate

    def gen_population(self):
        for _ in range(self.pop_size):
            route = np.random.permutation(self.vrproblem.get_sol_len())
            self.pop.append(route.astype(int))
        self.best_route = np.array([i for i in range(0, self.vrproblem.get_depot_num())])

    def eval_population(self):
        self.eval = np.array([self.vrproblem.calc_cost(route) for route in self.pop])

    def select_roulette(self):
        absolute_eval = np.abs(self.eval - self.eval.max() - 0.1)
        eval_div = absolute_eval / absolute_eval.sum()
        eval_div = eval_div.cumsum()
        new_pop = []
        for _ in range(self.pop_size):
            rand_val = np.random.random()
            for j in range(self.pop_size):
                if eval_div[j] >= rand_val:
                    new_pop.append(self.pop[j])
                    break
        self.pop = new_pop

    def mutate_one(self, entity):
        entity_length = len(entity)
        index_a = np.random.randint(0, entity_length)
        index_b = np.random.randint(0, entity_length)
        entity[index_a], entity[index_b] = entity[index_b], entity[index_a]
        return entity

    def selection_tournament(self):
        tour_size = int(len(self.pop) * 0.1)
        new_pop = []

        for _ in range(int(len(self.pop) / 2)):
            parent1 = np.random.choice(self.pop_size, size=1)[0]
            parent2 = np.random.choice([i for i in self.pop if i != parent1], size=1)[0]

            if self.vrproblem.calc_cost(parent1) < self.vrproblem.calc_cost(parent2):
                new_pop.append(parent1)
            else:
                new_pop.append(parent2)

        return new_pop


    def mutate_inverse(self, entity):
        entity_length = len(entity)
        start = np.random.randint(0, entity_length)
        end = np.random.randint(0, entity_length)
        if start > end:
            start, end = end, start
        while start < end:
            entity[start], entity[end] = entity[end], entity[start]
            start += 1
            end -= 1
        return entity

    def crossover_ox(self,parent1, parent2):
        length = len(parent1)
        start, end = sorted(np.random.choice(length, 2, replace=False))

        child = [None] * length
        seen = set(parent1[start:end+1])

        # Copy the segment from parent1 to the child
        child[start:end+1] = parent1[start:end+1]

        # Fill the remaining positions with values from parent2
        parent2_idx = 0
        for i in range(length):
            if child[i] is None:
                while parent2[parent2_idx] in seen:
                    parent2_idx += 1
                child[i] = parent2[parent2_idx]
                seen.add(parent2[parent2_idx])
                parent2_idx += 1

        return np.array(child)

    def crossover_population(self):
        num_parents = int(self.crossover_rate * self.pop_size)
        parents_indices = np.random.choice(self.pop_size, size=num_parents, replace=False)
        for i in range(0, num_parents, 2):
            parent1 = self.pop[parents_indices[i]]
            parent2 = self.pop[parents_indices[i + 1]]
            child1 = self.crossover_ox(parent1, parent2)
            child2 = self.crossover_ox(parent2, parent1)
            self.pop[parents_indices[i]] = child1
            self.pop[parents_indices[i + 1]] = child2

    def mutate_population(self):
        for i in range(self.pop_size):
            rand_val = np.random.random()
            if rand_val < self.mutation_chance:
                self.pop[i] = self.mutate_one(self.pop[i])
                self.pop[i] = self.mutate_inverse(self.pop[i])

    def get_best(self):
        for it, dist in enumerate(self.eval):
            if dist < self.shortest_dist:
                self.shortest_dist = dist
                self.best_route = self.pop[it]
        return self.shortest_dist

    def print_best(self):
        self.get_best()
        print(self.best_route)
        print(self.shortest_dist)
        self.vrproblem.print_routes(self.best_route)
        self.vrproblem.plot(self.best_route)

Optimize the crossover_ox function to increase time performance, ensure that returned child has no None values and no duplicates and only includes values present in one of the parents.

## Odpowiedzi

### Llama3

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start
    child = [None] * length
    child[start:end+1] = parent1[start:end+1]
    idx_child = (end + 1) % length
    idx_parent2 = (end + 1) % length
    for _ in range(length):
        if child[idx_child] is None:
            if parent2[idx_parent2] not in child:
                child[idx_child] = parent2[idx_parent2]
                idx_child = (idx_child + 1) % length
        idx_parent2 = (idx_parent2 + 1) % length
    return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start
    child = [parent1[i] if i >= start and i < end else parent2[i % length]
             for i in range(length)]
    return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start

    child = [None] * length
    child[start:end+1] = parent1[start:end+1]

    idx_child = (end + 1) % length
    for _ in range(length):
        if child[idx_child] is None:
            if parent2[idx_child] not in child:
                child[idx_child] = parent2[idx_child]
            idx_child = (idx_child + 1) % length

    return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length+1)

    if start > end:
        start, end = end, start

    child = []
    seen = set()

    for i in range(length):
        if start <= i < end:
            child.append(parent1[i])
        else:
            while True:
                value = parent2[i % length]
                if value not in seen:
                    child.append(value)
                    seen.add(value)
                    break

    return np.array(child)

In [ ]:
def crossover_ox(parent1, parent2):    
    length = len(parent1)
    start = np.random.randint(0, length) 
    end = np.random.randint(start, length+1)

    if start > end:
        start, end = end, start

    child = []
    seen = set()
    for i in range(length):
        if start <= i < end:
            child.append(parent1[i])
        else:
            while True:
                value = parent2[i % length]
                print(value)
                if value not in seen and value not in child:
                    child.append(value)
                    seen.add(value)
                    break

    return np.array(child)

### CodeLlama

In [ ]:
def crossover_ox(parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start
    child = [None] * length
    child[start:end+1] = parent1[start:end+1]
    idx_child = (end + 1) % length
    idx_parent2 = (end + 1) % length
    for _ in range(length):
        if child[idx_child] is None:
            if parent2[idx_parent2] not in child:
                child[idx_child] = parent2[idx_parent2]
                idx_child = (idx_child + 1) % length
        idx_parent2 = (idx_parent2 + 1) % length
    return np.array(child)

In [ ]:
def crossover_ox_improved(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start
    child1 = []
    child2 = []
    for i in range(start, end):
        if parent1[i] not in child1 and parent2[i] not in child2:
            child1.append(parent1[i])
            child2.append(parent2[i])
        elif parent1[i] in child1 and parent2[i] not in child2:
            child2.append(parent2[i])
        elif parent1[i] not in child1 and parent2[i] in child2:
            child1.append(parent1[i])
    for i in range(length):
        if child1[i] == None or child2[i] == None:
            break
    return child1, child2

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start
    child = [None] * length
    child[start:end+1] = parent1[start:end+1]
    idx_child = (end + 1) % length
    idx_parent2 = (end + 1) % length
    while True:
        if child[idx_child] is None and parent2[idx_parent2] not in child:
            child[idx_child] = parent2[idx_parent2]
            idx_child = (idx_child + 1) % length
            break
        idx_parent2 = (idx_parent2 + 1) % length
    return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start
    child = [None] * length
    child[start:end+1] = parent1[start:end+1]
    idx_child = (end + 1) % length
    idx_parent2 = (end + 1) % length
    while True:
        if child[idx_child] is None:
            child[idx_child] = parent2[idx_parent2]
            idx_child = (idx_child + 1) % length
            break
        idx_parent2 = (idx_parent2 + 1) % length
    return np.array(child)

In [ ]:
def crossover_ox(parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    child = []
    for i in range(start, end + 1):
        if parent1[i] not in child:
            child.append(parent1[i])
    for i in range(end + 1, length):
        if parent2[i] not in child:
            child.append(parent2[i])
    return np.array(child)

### ChatGPT

In [ ]:
def crossover_ox(parent1, parent2):
    length = len(parent1)
    start, end = sorted(np.random.choice(length, 2, replace=False))
    
    # Initialize the child with None values
    child = [None] * length
    seen = set()

    # Copy the segment from parent1 to the child and mark these values as seen
    for i in range(start, end + 1):
        child[i] = parent1[i]
        seen.add(parent1[i])

    # Fill the remaining positions in the child with values from parent2
    parent2_idx = 0
    for i in range(length):
        if child[i] is None:
            while parent2[parent2_idx] in seen:
                parent2_idx += 1
            child[i] = parent2[parent2_idx]
            seen.add(parent2[parent2_idx])
            parent2_idx += 1

    return np.array(child)

In [ ]:
def crossover_ox(parent1, parent2):
    length = len(parent1)
    start, end = sorted(np.random.choice(length, 2, replace=False))

    child = [None] * length
    seen = set(parent1[start:end+1])

    # Copy the segment from parent1 to the child
    child[start:end+1] = parent1[start:end+1]

    # Fill the remaining positions with values from parent2
    parent2_idx = 0
    for i in range(length):
        if child[i] is None:
            while parent2[parent2_idx] in seen:
                parent2_idx += 1
            child[i] = parent2[parent2_idx]
            seen.add(parent2[parent2_idx])
            parent2_idx += 1

    return np.array(child)

In [ ]:
def crossover_ox(parent1, parent2):
    length = len(parent1)
    start, end = sorted(np.random.randint(0, length, size=2))

    # Initialize the child with None values
    child = [None] * length

    # Copy the segment from parent1 to the child
    child[start:end + 1] = parent1[start:end + 1]

    # Fill the remaining positions with the elements from parent2 in order
    current_index = (end + 1) % length
    parent2_index = (end + 1) % length

    while None in child:
        if parent2[parent2_index] not in child:
            child[current_index] = parent2[parent2_index]
            current_index = (current_index + 1) % length
        parent2_index = (parent2_index + 1) % length

    return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start, end = sorted(np.random.randint(0, length, size=2))

    # Initialize the child with None values
    child = [None] * length

    # Copy the segment from parent1 to the child
    child[start:end + 1] = parent1[start:end + 1]

    # Fill the remaining positions with the elements from parent2 in order
    parent2_elements = [item for item in parent2 if item not in child]
    current_index = (end + 1) % length
    
    for element in parent2_elements:
        if child[current_index] is None:
            child[current_index] = element
            current_index = (current_index + 1) % length

    return np.array(child)

### Dolphin-Mixtral

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    if start > end:
        start, end = end, start
    child = [None] * length
    for i in range(length):
        if parent1[i] is not None and parent2[i] is not None:
            if parent1[i] != parent2[i]:  # Include only different values between parents
                child[start:end+1][np.random.randint(start, end+1)] = parent1[i]
        elif parent1[i] is not None:
            child[start:end+1][np.random.randint(start, end+1)] = parent1[i]
        elif parent2[i] is not None:
            child[start:end+1][np.random.randint(start, end+1)] = parent2[i]
    return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    child = [None] * length
    child[start:end+1] = parent1[start:end+1]
    idx_child = (end + 1) % length
    idx_parent2 = (end + 1) % length
    while None in child or len(np.unique(child)) < len(child):
        if child[idx_child] is None:
            if parent2[idx_parent2] not in child and parent2[idx_parent2] is not None:
                child[idx_child] = parent2[idx_parent2]
                idx_child = (idx_child + 1) % length
        idx_parent2 = (idx_parent2 + 1) % length
    return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
    # Check if any None values exist in the parents
    if None in [parent1, parent2]:
        return parent1, parent2

    # Create a set to store unique values from one of the parents
    unique_values = set()

    # Get the length of the longer parent route
    max_len = max(len(parent1), len(parent2))

    # Iterate through the indices to create child routes
    for i in range(max_len):
        if i < len(parent1) and i < len(parent2):
            value = parent1[i]
            unique_values.add(value)
        elif i < len(parent1) and i >= len(parent2):
            value = parent1[i]
            unique_values.add(value)
        elif i >= len(parent1) and i < len(parent2):
            # Choose a random value from the unique set if the index is out of range in the second parent
            value = choices(list(unique_values), k=1)[0]
        else:
            continue

        child = list(range(self.vrproblem.get_sol_len()))
        child[i] = value

    return tuple(child)

In [ ]:
def crossover_ox(parent1, parent2):
    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)
    child = [None] * length
    child[start:end+1] = parent1[start:end+1]
    
    idx_child = (end + 1) % length
    idx_parent2 = (end + 1) % length
    while None in child or len(set(child)) != len(child):
        if child[idx_child] is None:
            if parent2[idx_parent2] not in child:
                child[idx_child] = parent2[idx_parent2]
                idx_child = (idx_child + 1) % length
        idx_parent2 = (idx_parent2 + 1) % length
    return np.array(child)

### Gemini

In [ ]:
def crossover_ox(self, parent1, parent2):
        length = len(parent1)
        start = np.random.randint(0, length)
        end = np.random.randint(start, length)
        if start > end:
            start, end = end, start

        # Create a dictionary to map elements from parent2 to their indices
        parent2_map = {elem: i for i, elem in enumerate(parent2)}

        # Initialize child with elements from parent1 within the crossover range
        child = parent1.copy()
        child[start:end+1] = None

        # Fill the remaining part of the child from parent2 while avoiding duplicates
        idx_child = (end + 1) % length
        for _ in range(length):
            if child[idx_child] is None:
                # Check if the element from parent2 exists in child and hasn't been used yet
                if parent2_map.get(parent2[idx_child]) not in child:
                    child[idx_child] = parent2[idx_child]
                else:
                    # If the element is already used, find the next unused element in parent2
                    next_idx = (parent2_map[parent2[idx_child]] + 1) % length
                    while child[next_idx] is not None:
                        next_idx = (next_idx + 1) % length
                    child[idx_child] = parent2[next_idx]
            idx_child = (idx_child + 1) % length

        return np.array(child)

In [ ]:
def crossover_ox(self, parent1, parent2):
        length = len(parent1)
        start = np.random.randint(0, length)
        end = np.random.randint(start, length)
        if start > end:
            start, end = end, start

        # Create a dictionary to map elements from parent2 to their indices
        parent2_map = {elem: i for i, elem in enumerate(parent2)}

        # Initialize child with elements from parent1 (avoiding slice assignment)
        child = np.empty_like(parent1)  # Create an empty array with the same shape as parent1
        child[:start] = parent1[:start]
        child[end+1:] = parent1[end+1:]

        # Fill the remaining part of the child from parent2 while avoiding duplicates
        idx_child = (end + 1) % length
        for _ in range(length):
            if child[idx_child] is None:
                # Check if the element from parent2 exists in child and hasn't been used yet
                if parent2_map.get(parent2[idx_child]) not in child:
                    child[idx_child] = parent2[idx_child]
                else:
                    # If the element is already used, find the next unused element in parent2
                    next_idx = (parent2_map[parent2[idx_child]] + 1) % length
                    while child[next_idx] is not None:
                        next_idx = (next_idx + 1) % length
                    child[idx_child] = parent2[next_idx]
            idx_child = (idx_child + 1) % length

        return child

In [ ]:
def crossover_ox(self, parent1, parent2):
    """Performs Ordered Crossover (OX) on two parent routes, ensuring the child
       contains unique values from both parents without duplicates.

    Args:
        parent1 (np.ndarray): The first parent route.
        parent2 (np.ndarray): The second parent route.

    Returns:
        np.ndarray: The child route containing unique values from parents.
    """

    length = len(parent1)
    start = np.random.randint(0, length)
    end = np.random.randint(start, length)

    # Create a copy of the starting segment from parent1
    child = parent1[start:end + 1].copy()

    # Fill the remaining slots in the child using elements from parent2
    # that are not present in the child yet
    idx_child = (end + 1) % length
    idx_parent2 = (end + 1) % length
    while idx_child != start:
        if parent2[idx_parent2] not in child:
            child[idx_child] = parent2[idx_parent2]
        idx_child = (idx_child + 1) % length
        idx_parent2 = (idx_parent2 + 1) % length

    return child

In [ ]:
def crossover_ox(self, parent1, parent2):
  """
  This function performs Order Crossover (OX) on two parent routes,
  ensuring the child contains no None values, duplicates, and only values from parents.

  Args:
      parent1: A numpy array representing the first parent route.
      parent2: A numpy array representing the second parent route.

  Returns:
      A numpy array representing the child route.
  """
  length = len(parent1)
  start = np.random.randint(0, length)
  end = np.random.randint(start, length)

  # Initialize used_in_child with all possible values set to False.
  used_in_child = {v: False for v in range(length)}

  # Copy the segment from parent1 to the child.
  child = parent1.copy()
  child[start:end + 1] = parent1[start:end + 1]

  # Fill the remaining slots in the child using elements from parent2.
  idx_child = (end + 1) % length
  for idx_parent2 in range((end + 1) % length, length):
    if not used_in_child[parent2[idx_parent2]]:
      child[idx_child] = parent2[idx_parent2]
      used_in_child[parent2[idx_parent2]] = True
      idx_child = (idx_child + 1) % length

  # Fill the remaining slots at the beginning of the child using elements from parent2.
  for idx_parent2 in range(0, (end + 1) % length):
    if not used_in_child[parent2[idx_parent2]]:
      child[idx_child] = parent2[idx_parent2]
      used_in_child[parent2[idx_parent2]] = True
      idx_child = (idx_child + 1) % length

  return child

In [ ]:
def crossover_ox(parent1, parent2):
  length = len(parent1)
  start = np.random.randint(0, length)
  end = np.random.randint(start, length)

  # Copy the designated segment from parent1 to child
  child = [None] * length
  child[start:end+1] = parent1[start:end+1].copy()

  # Iterate through the child, filling in missing values from parent2
  idx_child = (end + 1) % length
  for _ in range(length):
    if child[idx_child] is None:
      # Find an unused value from parent2
      for val in parent2:
        if val not in child:
          child[idx_child] = val
          break
    idx_child = (idx_child + 1) % length

  return np.array(child)